---- Prévoir dans process ETL
Colonne "risque_gel_pluie"
- temp. <0
- weather_code : 61,63,65

Colonne "risque_gel_neige"
- temp. <0
- weather_code : 71,73,75 

Colonne "risque_neige_fondue"
- temp > 0
- weather_code : 71,73,75 

Prévoir que les transports arrivent à une heure précise et la météo est à heure pile --> arrondir l'heure du transport à l'heure la plus proche ?
Colonne 'timestamp_rounded'

"sunrise", "sunset"
--> effet saison

calendrier
--> via bibliothèque Python
--> jour férié
--> vacances
--> weekend

In [6]:
import pandas as pd
import json
import holidays
from datetime import datetime
import os

In [ ]:
# FONCTION 1 - ETL WEATHER TRANSFORMATION
def etl_weather_transformation(json_input):
    """Transforme le JSON brut Open-Meteo en DataFrame aplati avec logique Sunrise/Sunset"""
    with open(json_input, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Aplatissement Horaire
    df_h = pd.DataFrame(data['hourly'])
    df_h['timestamp'] = pd.to_datetime(df_h['time'])
    
    # Aplatissement Journalier (pour sunrise/sunset)
    df_d = pd.DataFrame(data['daily'])
    df_d['date'] = pd.to_datetime(df_d['time']).dt.date
    df_d['sunrise'] = pd.to_datetime(df_d['sunrise'])
    df_d['sunset'] = pd.to_datetime(df_d['sunset'])

    # Merge pour avoir sunrise/sunset sur chaque ligne horaire
    df_h['date'] = df_h['timestamp'].dt.date
    df = df_h.merge(df_d[['date', 'sunrise', 'sunset']], on='date', how='left')

    # Feature Engineering Météo : Gel et Lumière
    df['soleil_leve'] = ((df['timestamp'] >= df['sunrise']) & (df['timestamp'] <= df['sunset'])).astype(int)
    df['risque_gel_pluie'] = ((df['temperature_2m'] <= 0) & (df['precipitation'] > 0)).astype(int)
    df['risque_gel_neige'] = ((df['temperature_2m'] <= 0) & (df['snowfall'] > 0)).astype(int)
    df['neige_fondue'] = ((df['temperature_2m'] > 0) & (df['snowfall'] > 0)).astype(int)
    
    return df # à voir si on drop ici des colonnes inutiles ? ou alors est dans neon DB qu'on fait le tri ?

# FONCTION 2 - ENRICHISSEMENT CALENDRIER
def enrich_calendar_features(df_calendrier_suedois):
    """Ajoute des features calendrier à la météo"""
    sweden_holidays = holidays.CountryHoliday('SE')

    # Bases (heures, jour, weekend, ferie)
    df_calendrier_suedois['year'] = df_calendrier_suedois['timestamp'].dt.year
    df_calendrier_suedois['month'] = df_calendrier_suedois['timestamp'].dt.month # identifier les saisons
    df_calendrier_suedois['day'] = df_calendrier_suedois['timestamp'].dt.day
    df_calendrier_suedois['day_of_week'] = df_calendrier_suedois['timestamp'].dt.dayofweek
    df_calendrier_suedois['est_weekend'] = df_calendrier_suedois['day_of_week'].isin([5, 6]).astype(int) # ISO8601 0=Lundi, 1=Mardi,... 5=Samedi, 6=Dimanche
    df_calendrier_suedois['est_jour_ferie'] = df_calendrier_suedois['timestamp'].dt.date.apply(lambda x: 1 if x in sweden_holidays else 0)
     
    # Logique vacances scolaires (simplifiée ici)
    def est_vacances_scolaires(date):

        week = date.isocalendar()[1]
        month = date.month
        day = date.day
        
        # Sportlov (Semaine 9)
        if week == 9: return 1
        # Pasklov (Semaine 15 - approximation courante)
        if week == 15: return 1
        # Sommarlov (Juin semaine 24 à Août semaine 33)
        if 24 <= week <= 33: return 1
        # Höstlov (Semaine 44)
        if week == 44: return 1
        # Jullov (Noël/Nouvel an)
        if (month == 12 and day >= 21) or (month == 1 and day <= 8): return 1
        
        return 0

    df_calendrier_suedois['vacances_scolaires'] = df_calendrier_suedois['timestamp'].apply(est_vacances_scolaires)
    
    return df_calendrier_suedois

# FONCTION 3 - PIPELINE COMPLÈTE
def run_full_pipeline(input_filename, output_filename):
    """"Execute la pipeline ETL complète : transformation météo + enrichissement calendrier + sauvegarde"""
    # Construction du chemin vers le dossier voisin
    # '..' remonte dans le dossier d'avant
    base_path = os.path.abspath("") # dirname(__file__) pour passage script # Dossier courant (test_etl)
    input_path = os.path.join(base_path, "..", "test_call_api", input_filename)
    output_path = os.path.join(base_path, output_filename)

    if not os.path.exists(input_path):
        print(f"Erreur : Le fichier {input_path} est introuvable.")
        return

    df_step_meteo = etl_weather_transformation(input_path)
    df_final = enrich_calendar_features(df_step_meteo)


    # Sauvegarde locale avant Neon 
    df_final.to_parquet(output_path, index=False)
    print(f" Transformation terminée. Fichier sauvegardé sous {output_path}")
    
    return df_final

# --- EXÉCUTION ---
# Exemple d'exécution de la pipeline complète
run_full_pipeline(
    input_filename="weather_stockholm_archive.json",
    output_filename="weather_calendrier_stockholm_archive_transformed.parquet"
)

run_full_pipeline(
    input_filename="weather_stockholm_today+7j.json",
    output_filename="weather_calendrier_stockholm_today+7j_transformed.parquet"
) 

 Transformation terminée. Fichier sauvegardé sous d:\Profils\NLefort\Desktop\JEDHA\delay-forecast\test_etl\weather_calendrier_stockholm_archive_transformed.parquet
 Transformation terminée. Fichier sauvegardé sous d:\Profils\NLefort\Desktop\JEDHA\delay-forecast\test_etl\weather_calendrier_stockholm_today+7j_transformed.parquet


,time,temperature_2m,precipitation,rain,wind_speed_10m,snowfall,wind_gusts_10m,weather_code,cloud_cover,shortwave_radiation,...,risque_gel_pluie,risque_gel_neige,neige_fondue,year,month,day,day_of_week,est_weekend,est_jour_ferie,vacances_scolaires
0,2025-12-22T00:00,0.8,0.0,0.0,8.6,0.0,13.7,0,0,0.0,...,0,0,0,2025,12,22,0,0,0,1
1,2025-12-22T01:00,0.6,0.0,0.0,10.1,0.0,16.9,0,4,0.0,...,0,0,0,2025,12,22,0,0,0,1
2,2025-12-22T02:00,0.4,0.0,0.0,10.1,0.0,17.3,1,20,0.0,...,0,0,0,2025,12,22,0,0,0,1
3,2025-12-22T03:00,0.1,0.0,0.0,9.0,0.0,16.9,3,80,0.0,...,0,0,0,2025,12,22,0,0,0,1
4,2025-12-22T04:00,-0.3,0.0,0.0,7.2,0.0,13.0,1,28,0.0,...,0,0,0,2025,12,22,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,2025-12-29T19:00,-0.1,0.0,0.0,23.0,0.0,50.4,0,0,0.0,...,0,0,0,2025,12,29,0,0,0,1
188,2025-12-29T20:00,-0.2,0.0,0.0,22.8,0.0,50.0,0,0,0.0,...,0,0,0,2025,12,29,0,0,0,1
189,2025-12-29T21:00,-0.2,0.0,0.0,22.5,0.0,50.0,0,0,0.0,...,0,0,0,2025,12,29,0,0,0,1
190,2025-12-29T22:00,-0.2,0.0,0.0,22.3,0.0,50.0,0,0,0.0,...,0,0,0,2025,12,29,0,0,0,1


# NEON DB

--> séparer le stockage en deux
- un table historique (qui contient les retards) pour l'entraînement
- une table prévisions qui contient uniquement les variables météo à venir

- la table historique entraîne mon modèle
- mes données prévisions utilisents ce modèle pour générer des alertes de retards en temps réel

In [ ]:
from sqlalchemy import create_engine
import os

In [ ]:
def load_to_neon_db(df_calendrier_suedois):
    """Envoi vers Neon DB
    mode="replace" : écrase la table existante --> pour les prévisions
    mode="append" : ajoute les nouvelles données à la table existante --> pour l'historique"""

    db_url = os.getenv("NEON_DB_URL")
    engine = create_engine(db_url)

    # Chargement dans la table 'weather_calendar'
    df_calendrier_suedois.to_sql('weather_calendar', engine, if_exists='append', index=False)